v0: DBPIA, Scienceon Rule계산(Yop + co_authorship + keyword)
v1: title 조건 추가, jaro_winkler_metric 추가
v2: site 조건 변경
v3: Rule계산(joc + yop + title_cossim + abstract_cossim) 추가, TF-IDF 추가, 교신저자 비교
v4: co_authorship 비율추가
v5: site 조건 수정, 함수/변수이름 변경, filter 함수 추가, title_cossim + abstract_cossim 삭제, check_college추가
v6: co_author_ratio 조건 추가, isEnglishOrKorean 추가, filter에 매개변수 inst 추가 및 return instbool 추가, joc에 값이 다를 경우 0으로 변경

In [3]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
import jaro
import re

def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>1 else "e"

def check_college(univ0):
    branch_set = ['성균관대학교', '건국대학교', '한양대학교']
    univName = client['PUBLIC']['CollegeName']
    univ1 = re.sub("산학협력단|병원","",univ0)
    univ2 = re.sub("대학교","대학교 ",univ1)

    try:
        if isEnglishOrKorean(univ0) == 'e':
            univ0 = univ0.upper()
            univ0 = univ0.replace('.', ',')
            univ = univ0.split(', ')
        else:
            univ = univ2.replace(",", "").split()
            univ = list(set(univ))   
            
        for uni in univ:
            if uni in branch_set:
                if ("ERICA" or "에리카") in univ0:
                    univ[univ.index("한양대학교")] = "한양대학교(ERICA캠퍼스)"
                elif ("글로컬" or "GLOCAL") in univ0:
                    univ[univ.index("건국대학교")] = "건국대학교 GLOCAL(글로컬)캠퍼스"
                elif "자연과학캠퍼스" in univ0:
                    univ[univ.index("성균관대학교")] = "성균관대학교(자연과학캠퍼스)"

        univs = '{"$or": ['
        for u in range(len(univ)):
            if univ[-1] == univ[u]:
                univs += '{"inputName": "' + univ[u] + '"}'
            else:
                univs += '{"inputName": "' + univ[u] + '"}, '
        univs += ']}'

        univ_query = univName.find_one(eval(univs))

        if univ_query is None:
            print("Search inst None")
            return univ0, False
        else:
            return univ_query['originalName'], True
        
    except SyntaxError as e:
        print(e)
        print(univ0)
        return univ0, False

def filter(site, inst, rawdata):
    instbool = False
    if site == 'NTIS' :
        inst, instbool = check_college(inst)
        inst = rawdata['ldAgency']
        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""
    else :
        inst, instbool = check_college(inst)
        inst = rawdata['author_inst'].split(";")[rawdata['author'].split(";").index(name)]
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        
        if rawdata['paper_keyword'] == [] or rawdata['paper_keyword'] == 'None':
            keyword = []
        elif len(rawdata['paper_keyword']) > 1:
            for i in range(0, len(rawdata['paper_keyword'])):
                keyword = []
                keyword.append(rawdata['paper_keyword'][i].replace(" ", "").split("."))            
        else:
            keyword = rawdata['paper_keyword'].replace(" ", "").split(".")

        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']

    return inst, coauthor, year, keyword, journal, conference, title, instbool

def Secondary_filter(name, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst_simi = 0
    weight = 0
    joc = 0

    inst1, coauthor1, year1, keyword1, journal1, conference1, title1, instbool1 = filter(site1, inst1, raw_one1)
    inst2, coauthor2, year2, keyword2, journal2, conference2, title2, instbool2 = filter(site2, inst2, raw_one2)

    if instbool1 and instbool2:
        if inst1 == inst2:
            inst_simi = 1
    else:
        inst_simi = jaro.jaro_winkler_metric(inst1, inst2)
        
    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 or inst_simi >= 0.8: #or mng1 == mng2: # 
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else 0
    else:
        if inst_simi >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [5]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '이상원'
site1 = 'Scienceon'
site2 = 'KCI'
input_paper1 = ObjectId('61cbfa95597f83aefb455c3a')
input_paper2 = ObjectId('61cbfb6bf72c82d20d455ca1')
inst1 = ''
inst2 = ''

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'KCI' :
    site_raw1 = client['KCI']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'KCI' :
    site_raw2 = client['KCI']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

Secondary_filter(name, site1, inst1, input_raw1, site2, inst2, input_raw2)

4